# 2019-nCoV mapping

##### @sungml92
##### data source: Coronavirus 2019-nCoV Global Cases by Johns Hopkins CSSE
##### Google sheet api and google drive api code reference: http://bit.ly/386nKuk, http://bit.ly/2tDG12Z

### Coronavirus 2019-nCoV John Hopkins CSSE data import with google api

#### I. import goolge sheet using google api

In [15]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds']#,'https://www.googleapis.com/auth/drive']

# credential is anonymised for privacy
cred_path_and_file = 'your_credential_please'

creds = ServiceAccountCredentials.from_json_keyfile_name(cred_path_and_file,scope)
client = gspread.authorize(creds)

# lazy approach retrieve by url
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1wQVypefm946ch4XDp37uZ-wartW4V7ILdg-qYiDXUHM/htmlview?usp=sharing&sle=true#')
# or less lazy approach 
# sheet = client.open_by_key('1wQVypefm946ch4XDp37uZ-wartW4V7ILdg-qYiDXUHM')

# enter the number of the worksheet you wish to access. The number starts at 0
# by title also works when prompt worksheet = sheet.worksheet("name of work sheet")
# get the most recent record from url
worksheet = sheet.get_worksheet(0)




#### II. Beautify json output and json-table convert

In [22]:
import pprint
import pandas as pd

# check if data is correct
pp = pprint.PrettyPrinter()
nCoV = worksheet.get_all_records()
#pp.pprint(nCoV)

# open sheet with pd.DataFrame()
j2df = pd.DataFrame(nCoV)


    Confirmed        Country/Region  Deaths    Last Update    Province/State  \
0       22112        Mainland China     618   2/6/20 23:23             Hubei   
1         970        Mainland China       0   2/6/20 12:43         Guangdong   
2         954        Mainland China       0   2/6/20 10:53          Zhejiang   
3         851        Mainland China       2   2/6/20 11:23             Henan   
4         711        Mainland China       0   2/6/20 13:13             Hunan   
5         600        Mainland China       0    2/6/20 2:33           Jiangxi   
6         591        Mainland China       0   2/6/20 13:33             Anhui   
7         411        Mainland China       2   2/6/20 23:33         Chongqing   
8         373        Mainland China       0    2/6/20 8:03           Jiangsu   
9         347        Mainland China       0    2/6/20 7:53          Shandong   
10        344        Mainland China       1    2/7/20 0:53           Sichuan   
11        274        Mainland China     

#### III. match latitude and longitude for Province and Country

In [131]:
from geopy.geocoders import Nominatim

j2df_v2 = j2df

for index in j2df.index:
    Country = j2df.iloc[index,1]
    State = j2df.iloc[index,4]
    
    # Avoid un-recognized geo-location format
    if State == Country:
        State = ""
    if Country == "Mainland China":
        Country = Country.split(" ")[1]

    place = State if len(State) != 0 else Country
    
    print("finding geolocation of %s %s" % (State, Country))
    try:
        geolocator = Nominatim(user_agent="geoloc_nCoV")
        location = geolocator.geocode(State+','+Country)
        print("latitude and longitude for %s %s is following" % (State, Country))
        print((location.latitude, location.longitude))
        
    except:
        print("cannot understand input State/Country, please check your format")
        break
    
    # add column for latitude and longitude
    print("adding latitude and longitude to new dataframe")
    j2df_v2.loc[j2df_v2.index[index], 'latitude'] = location.latitude
    j2df_v2.loc[j2df_v2.index[index], 'longitude'] = location.longitude
    j2df_v2.loc[j2df_v2.index[index], 'location'] = place
    
    

finding geolocation of Hubei China
latitude and longitude for Hubei China is following
(31.15172525, 112.87832224656043)
adding latitude and longitude to new dataframe
finding geolocation of Guangdong China
latitude and longitude for Guangdong China is following
(23.1357694, 113.1982688)
adding latitude and longitude to new dataframe
finding geolocation of Zhejiang China
latitude and longitude for Zhejiang China is following
(29.0000001, 119.9999999)
adding latitude and longitude to new dataframe
finding geolocation of Henan China
latitude and longitude for Henan China is following
(34.0000001, 113.9999999)
adding latitude and longitude to new dataframe
finding geolocation of Hunan China
latitude and longitude for Hunan China is following
(27.9995878, 112.009538)
adding latitude and longitude to new dataframe
finding geolocation of Jiangxi China
latitude and longitude for Jiangxi China is following
(28.0, 116.0)
adding latitude and longitude to new dataframe
finding geolocation of Anhu

latitude and longitude for  Russia is following
(64.6863136, 97.7453061)
adding latitude and longitude to new dataframe
finding geolocation of  UK
latitude and longitude for  UK is following
(54.7023545, -3.2765753)
adding latitude and longitude to new dataframe
finding geolocation of Chicago, IL US
latitude and longitude for Chicago, IL US is following
(41.8755616, -87.6244212)
adding latitude and longitude to new dataframe
finding geolocation of San Benito, CA US
latitude and longitude for San Benito, CA US is following
(36.6248089, -121.1177379)
adding latitude and longitude to new dataframe
finding geolocation of Santa Clara, CA US
latitude and longitude for Santa Clara, CA US is following
(37.2333253, -121.6846349)
adding latitude and longitude to new dataframe
finding geolocation of  Belgium
latitude and longitude for  Belgium is following
(50.6402809, 4.6667145)
adding latitude and longitude to new dataframe
finding geolocation of  Cambodia
latitude and longitude for  Cambodia i

#### Now time to map! Happy mapping

In [134]:
import folium
import pandas as pd
    
# Make an empty map
m = folium.Map([20, 20], zoom_start=2)

# I can add marker one by one on the map
for i in range(0,len(j2df_v2)):
    folium.Circle(
    location=[j2df_v2.iloc[i]['latitude'],j2df_v2.iloc[i]['longitude']],
    popup=j2df_v2.iloc[i]['location'],
    radius=float(j2df_v2.iloc[i]['Confirmed']*10),
    color='crimson',
    fill=True,
    fill_color='crimson'
   ).add_to(m)

# Save it as html
m.save('2019-nCoV_map_JHUCSSEdata_02072020.html')
